# Sparkify Churn Prediction

## 1. Load and clean dataset
Loading and cleaning the dataset, checking for invalid or missing data - for example, records without user IDs or session IDs.

In [148]:
# Import libraries

from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, udf, from_unixtime, substring, instr, encode, decode, split
from pyspark.sql.types import LongType, IntegerType, StringType, BooleanType

import pandas as pd

from user_agents import parse

import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
from axes import Axes

In [2]:
# Correct autocomplete not working

%config Completer.use_jedi = False

In [3]:
# Create spark session

spark = SparkSession \
    .builder \
    .appName('Sparkify') \
    .getOrCreate()

In [4]:
# Show link to spark UI at localhost:4040

spark

In [5]:
# Read sparkify dataset

filepath = 'mini_sparkify_event_data.json'
users_log = spark.read.json(filepath)

# Full dataset (12 Gb, 20 million rows)
# event_data = 's3n://udacity-dsnd/sparkify/sparkify_event_data.json'

# Mini dataset (128 Mb, 200 thousand rows, 1% of the full dataset)
# event_data = 's3n://udacity-dsnd/sparkify/mini_sparkify_event_data.json'

In [6]:
print('users_log shape:', (users_log.count(), len(users_log.columns)))

users_log shape: (286500, 18)


In [7]:
users_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [8]:
# Adjust the data types

users_log = users_log.withColumn('registration', from_unixtime(col('registration') / 1000))
users_log = users_log.withColumn('ts', from_unixtime(col('ts') / 1000))
users_log = users_log.withColumn('userId', col('userId').cast(LongType()))

In [9]:
# Show example of string with wrong encoding

record = '''userId == 30
            and sessionId == 29
            and itemInSession == 68'''

encoding_example = users_log.where(record).collect()[0][0]

print('String should be:\t Björk')
print('String currently is:\t {}'.format(encoding_example))

String should be:	 Björk
String currently is:	 BjÃÂ¶rk


In [10]:
def encode_decode_column(df, column, encoding, decoding):
    """Encode a column from a dataframe and then decode it.
    
    Parameters:
        df (Spark dataframe): The dataframe that contains the column.
        column (String): The name of the column, from the df, to perform the encode and decode.
        encoding (string): The charset of the encoding (one of 'US-ASCII', 'ISO-8859-1', 'UTF-8', 'UTF-16BE', 'UTF-16LE', 'UTF-16').
        decoding (string): The charset of the decoding (one of 'US-ASCII', 'ISO-8859-1', 'UTF-8', 'UTF-16BE', 'UTF-16LE', 'UTF-16').
    
    Returns:
        df (Spark dataframe):
        
    Example:
        df = encode_decode_column(df, 'column_name', 'ISO-8859-1', 'UTF-8')
    """
    
    df = df.withColumn(column, encode(column, encoding))
    df = df.withColumn(column, decode(column, decoding))
    
    return df

In [11]:
date_columns = ['registration', 'ts']
categorical_columns = list(filter(lambda c: c[1] == 'string', users_log.dtypes))
categorical_columns = [c[0] for c in categorical_columns if c[0] not in date_columns]

In [12]:
# Fix the wrong encoding for the columns that are strings,
# and in order to retrieve the correct characters the encode-decode process must be done twice.

for column in categorical_columns:
    users_log = encode_decode_column(users_log, column, 'ISO-8859-1', 'UTF-8')
    users_log = encode_decode_column(users_log, column, 'ISO-8859-1', 'UTF-8')

In [13]:
# Test if the previous example is correct now

encoding_example_fixed = users_log.where(record).collect()[0][0]

print('String should be:\t Björk')
print('String currently is:\t {}'.format(encoding_example_fixed))

String should be:	 Björk
String currently is:	 Björk


In [12]:
users_log.limit(5).toPandas().head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Martha Tilston,Logged In,Colin,M,50,Freeman,277.89016,paid,"Bakersfield, CA",PUT,NextSong,2018-09-28 19:22:42,29,Rockpools,200,2018-09-30 21:01:57,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
1,Five Iron Frenzy,Logged In,Micah,M,79,Long,236.09424,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,2018-09-30 15:20:30,8,Canada,200,2018-09-30 21:03:00,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
2,Adam Lambert,Logged In,Colin,M,51,Freeman,282.82730,paid,"Bakersfield, CA",PUT,NextSong,2018-09-28 19:22:42,29,Time For Miracles,200,2018-09-30 21:06:34,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
3,Enigma,Logged In,Micah,M,80,Long,262.71302,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,2018-09-30 15:20:30,8,Knocking On Forbidden Doors,200,2018-09-30 21:06:56,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
4,Daft Punk,Logged In,Colin,M,52,Freeman,223.60771,paid,"Bakersfield, CA",PUT,NextSong,2018-09-28 19:22:42,29,Harder Better Faster Stronger,200,2018-09-30 21:11:16,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30


In [14]:
numerical_columns = list(filter(lambda c: c[1] == 'bigint' or c[1] == 'double', users_log.dtypes))
numerical_columns = list(item[0] for item in numerical_columns)

In [15]:
summary = users_log.select(numerical_columns).describe().toPandas().set_index('summary')
summary.index.name = None
summary

,itemInSession,length,sessionId,status,userId
count,286500,228108,286500,286500,278154
mean,114.41421291448516,249.11718197783722,1041.526554973822,210.05459685863875,59682.02278593872
stddev,129.76726201141085,99.23517921058324,726.7762634630834,31.50507848842202,109091.94999910519
min,0,0.78322,1,200,2
max,1321,3024.66567,2474,404,300025


In [ ]:
# ADD BACK THE FUNCTION TO SHOW THE PERIOD OF THE DATE FIELDS

In [16]:
users_log_valid = users_log.dropna(how='any', subset=['userId', 'sessionId'])

old_rows, old_columns = users_log.count(), users_log.columns
new_rows, new_columns = users_log_valid.count(), users_log_valid.columns

print('users_log shape:\t', (old_rows, len(old_columns)))
print('users_log_valid shape:\t', (new_rows, len(new_columns)))
print('{} rows with empty user and session IDs removed'.format(old_rows - new_rows))

users_log shape:	 (286500, 18)
users_log_valid shape:	 (278154, 18)
8346 rows with empty user and session IDs removed


In [17]:
distinct_user_ids = users_log_valid.dropDuplicates(['userId']).count()
print('Distinct user IDs:', distinct_user_ids)

Distinct user IDs: 225


## 2. Exploratory Data Analysis

### Defining churn
Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

In [206]:
# Add churn

flag_cancellation_event = udf(lambda x: 1 if x == 'Cancellation Confirmation' else 0, IntegerType())
users_log_eda = users_log_valid.withColumn('churn', flag_cancellation_event('page'))

In [ ]:
# 1.  [ok] Existem páginas next songs que possuem o artista e a música nulos?
# 2.  [+-] Tirar o gráfico de contagem de gênero e colocar um que mostre os gêneros removendo as duplicatas de userId
# 3.  [+-] Mostrar a localidade agrupada por status e removida as duplicatas
# 4.  [  ] Criar um gráfico para mostrar a plataforma dos usuários
# 5.  [  ] Criar um gráfico para mostrar o browser dos usuários
# 6.  [  ] Mostrar o level removendo as duplicatas de userId, e mostrar como ele evolui no tempo para paid e free
# 7.  [+-] Substituir o gráfico do count de artista pelo artista mais tocado
# 8.  [+-] Substituir o gráfico do count de música pela música mais tocada
# 9.  [  ] Mostrar a curva de acquisition
# 10. [  ] Mostrar a curva de churn
# 11. [ok] Criar uma coluna com a cohort
# 12. [  ] Mostrar a curva de churn por gênero, cohort, músicas escutadas na plataforma, level, localização.
#          Colocar em valores aboslutos e percentuais.

In [ ]:
# TRANSFORM THIS CELL TO PLOT ALL THE NUMERICAL COLUMNS

# plt.figure(figsize=(15, 7), dpi=500)

# plt.hist(users_log_valid.select('userID').dropDuplicates().toPandas(), bins=12)
# plt.xlabel('User ID')
# plt.ylabel('Users')
# plt.title('Distribution of user IDs')

# plt.show();

In [136]:
# Add the cohort

users_log_eda = users_log_eda.withColumn('cohort', substring('registration', 1, 7))

In [137]:
# Add the state and city

users_log_eda = users_log_eda.withColumn('city', split('location', ',')[0])
users_log_eda = users_log_eda.withColumn('state', split('location', ',')[1])
users_log_eda = users_log_eda.drop('location')

In [138]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)  # REMOVE THIS LINE LATER

In [ ]:
group_by_count_artist = users_log_valid.select('artist').where('artist is not null').groupby('artist').count()
group_by_count_artist = group_by_count_artist.toPandas().sort_values(by='count')
group_by_count_artist

In [ ]:
group_by_count_song = users_log_valid.select('song').where('song is not null').groupby('song').count()
group_by_count_song = group_by_count_song.toPandas().sort_values(by='count')
group_by_count_song

In [ ]:
group_by_count_gender = users_log_valid.select('userID', 'gender').dropDuplicates(['userId']).groupby('gender').count()
group_by_count_gender = group_by_count_gender.toPandas().sort_values(by='count')
group_by_count_gender

In [ ]:
def create_date_table(start='2018-03-18', end='2018-12-02'):
    df = pd.DataFrame({'date': pd.date_range(start, end)})
    df['week'] = df['date'].dt.isocalendar().week
    df['month'] = df['date'].dt.month
    return df

In [83]:
date_table = create_date_table()
date_table

,date,week,month
0,2018-03-18,11,3
1,2018-03-19,12,3
2,2018-03-20,12,3
3,2018-03-21,12,3
4,2018-03-22,12,3
...,...,...,...
255,2018-11-28,48,11
256,2018-11-29,48,11
257,2018-11-30,48,11
258,2018-12-01,48,12


In [ ]:
windowval = Window.partitionBy(['userId', 'page']).orderBy(desc('ts')).rowsBetween(Window.unboundedPreceding, currentRow())
# windowval = Window.partitionBy('userId').orderBy(desc('ts')).rangeBetween(Window.unboundedPreceding, 0)
# user_log_valid = user_log_valid.withColumn("phase", Fsum("downgraded").over(windowval))

In [ ]:
# Earliest registration: 2018-03-18 10:44:35
# Latest registration: 2018-11-26 13:49:14
# Earliest ts: 2018-09-30 21:01:57
# Latest ts: 2018-12-02 23:11:16

### User agent parser

In [187]:
users_log_eda.select('userAgent').dropDuplicates(['userAgent']).limit(10).toPandas().head(10)

,userAgent
0,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"""
1,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"""
2,Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:31.0) Gecko/20100101 Firefox/31.0
3,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"""
4,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"""
5,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.7; rv:31.0) Gecko/20100101 Firefox/31.0
6,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0
7,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0
8,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)
9,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"""


In [207]:
get_browser     = udf(lambda x: parse(x).browser.family, StringType())
get_os          = udf(lambda x: parse(x).os.family, StringType())
get_device      = udf(lambda x: parse(x).device.family, StringType())
get_is_phone    = udf(lambda x: 1 if parse(x).is_mobile else 0, IntegerType())
get_is_tablet   = udf(lambda x: 1 if parse(x).is_tablet else 0, IntegerType())
get_is_computer = udf(lambda x: 1 if parse(x).is_pc else 0, IntegerType())

In [208]:
users_log_eda = users_log_eda.withColumn('browser', get_browser('userAgent'))
users_log_eda = users_log_eda.withColumn('os', get_os('userAgent'))
users_log_eda = users_log_eda.withColumn('device', get_device('userAgent'))
users_log_eda = users_log_eda.withColumn('isPhone', get_is_phone('userAgent'))
users_log_eda = users_log_eda.withColumn('isTablet', get_is_tablet('userAgent'))
users_log_eda = users_log_eda.withColumn('isComputer', get_is_computer('userAgent'))

In [209]:
users_log_eda \
    .select('userAgent', 'browser', 'os', 'device', 'isPhone', 'isTablet', 'isComputer') \
    .dropDuplicates(['userAgent']) \
    .limit(10) \
    .toPandas() \
    .head(10)

,userAgent,browser,os,device,isPhone,isTablet,isComputer
0,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",Chrome,Mac OS X,Mac,0,0,1
1,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",Chrome,Windows,Other,0,0,1
2,Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:31.0) Gecko/20100101 Firefox/31.0,Firefox,Ubuntu,Other,0,0,1
3,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",Chrome,Mac OS X,Mac,0,0,1
4,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36""",Chrome,Mac OS X,Mac,0,0,1
5,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.7; rv:31.0) Gecko/20100101 Firefox/31.0,Firefox,Mac OS X,Mac,0,0,1
6,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0,Firefox,Windows,Other,0,0,1
7,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0,Firefox,Windows,Other,0,0,1
8,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0),IE,Windows,Other,0,0,1
9,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",Chrome,Windows,Other,0,0,1


### Exploring data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [39]:
def group_by_count_column(df, column, n_labels=20, n_labels_chars=20):
    """Perform a group by and count the values from a desired column.
    
    Parameters:
        df (Spark dataframe): The dataframe that contains the column.
        column (String): The name of the column, from the df, to perform the group by and count its values.
    
    Returns:
        x:
        y:
    """
    
    group_by_count = df.select(column).groupby(column).count().toPandas().sort_values(by='count')
    
    x = group_by_count[column][-n_labels:]
    x = x.apply(lambda s: 'None' if s is None else s)
    x = x.apply(lambda s: s[:n_labels_chars] + '...' if len(s) >= n_labels_chars else s)
    y = group_by_count['count'][-n_labels:]
    
    return x, y

In [40]:
x, y = [], []
for column in categorical_columns:
    x_series, y_series = group_by_count_column(users_log_valid, column)
    x.append(x_series)
    y.append(y_series)

In [42]:
plt.style.use('ggplot')

fig = plt.figure(figsize=(12, 22), dpi=500)
nrows, ncols = (5, 2)
grid = gs.GridSpec(nrows=nrows, ncols=ncols)

titles = ['Artist', 'Auth', 'First name', 'Gender', 'Last name', 'Level', 'Location', 'Method', 'Page', 'Song']
nbins = [10, 10, 10, 6, 10, 10, 10, 10, 10, 10]
rows = [r for r in range(nrows) for _ in (0, 1)]
cols = [0 if c % 2 == 0 else 1 for c in range(nrows*ncols)]

fig.text(0.572, 1, 'Count of the top 20 classes for each categorical feature', ha='center', va='top', size=15, weight='bold')

for i in range(nrows*ncols):
    Axes(x[i], y[i], titles[i], nbins=nbins[i]).subplot(grid, rows[i], cols[i])
    
plt.tight_layout(h_pad=2, w_pad=2, rect=(0, 0, 1, 0.98))
plt.show();

In [ ]:
plt.style.use('ggplot')

fig = plt.figure(figsize=(12, 22), dpi=500)
grid = gs.GridSpec(nrows=5, ncols=2)

fig.text(0.571, 1, 'Count of the top 20 classes for each categorical feature', ha='center', va='top', size=15, weight='bold')

Axes(x_artist,   y_artist,   'Artist',    nbins=2).subplot(grid, 0, 0)
Axes(x_auth,     y_auth,     'Auth'              ).subplot(grid, 0, 1)
Axes(x_first,    y_first,    'First name'        ).subplot(grid, 1, 0)
Axes(x_gender,   y_gender,   'Gender',    nbins=6).subplot(grid, 1, 1)
Axes(x_last,     y_last,     'Last name'         ).subplot(grid, 2, 0)
Axes(x_level,    y_level,    'Level'             ).subplot(grid, 2, 1)
Axes(x_location, y_location, 'Location'          ).subplot(grid, 3, 0)
Axes(x_method,   y_method,   'Method'            ).subplot(grid, 3, 1)
Axes(x_page,     y_page,     'Page'              ).subplot(grid, 4, 0)
Axes(x_song,     y_song,     'Song',      nbins=2).subplot(grid, 4, 1)

plt.tight_layout(h_pad=2, w_pad=2, rect=(0, 0, 1, 0.98))
plt.show();

## 3. Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.

- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

## 4. Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

## 5. Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.